In [ ]:
import numpy as np
import pandas as pd

roles_wide = pd.read_csv("combined_V1_V2.csv",  na_filter=False, index_col=0)

properties_list = ['awareness', 'change_of_location', 'change_of_state',
       'change_of_possession', 'existed_after', 'existed_before',
       'existed_during', 'instigation', 'sentient', 'volition']

roles_wide.rename({"Sentence": "sentence", "Arg.Phrase": "arg"}, axis=1, inplace=True)


In [3]:
def make_labels(x):
    x = np.array(x)
    labels = np.zeros(len(properties_list))
    labels[np.where(x > 3)] = 1
    # labels[np.where(x <= 2)] = 0
    # labels[np.where(x >= 4)] = 2
    return labels

roles_wide["combined_labels"] = roles_wide["combined_labels"].apply(lambda x: eval(x))
roles_wide['labels'] = roles_wide["combined_labels"].apply(make_labels)

roles_wide[properties_list] = roles_wide['labels'].tolist()

roles_wide.head()


,Sentence.ID,Roleset,Gram.Func,sentence,Predicate,Split,arg,Arg.Stripped,Arg,arg_idx,...,awareness,change_of_location,change_of_state,change_of_possession,existed_after,existed_before,existed_during,instigation,sentient,volition
0,0003_21,impose.01,nsubj,"In July , a gradual ban was imposed by the Env...",was imposed by,test,a gradual ban,a ban,1,"(3, 6)",...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0003_21,impose.01,dobj,"In July , a gradual ban was imposed by the Env...",was imposed by,train,the Environmental Protection Agency,the Environmental Protection Agency,0,"(9, 13)",...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
2,0003_29,have.03,nsubj,No bearing on our work force today is had by it .,is had by,train,No bearing on our work force today,no bearing,1,"(0, 7)",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0003_29,have.03,dobj,No bearing on our work force today is had by it .,is had by,test,it,It,0,"(10, 11)",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0003_9,lead.02,nsubj,A team of researchers from the National Cancer...,was led by,train,A team of researchers from the National Cancer...,a team,1,"(0, 19)",...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [4]:
roles_wide.columns


Index(['Sentence.ID', 'Roleset', 'Gram.Func', 'sentence', 'Predicate', 'Split',
       'arg', 'Arg.Stripped', 'Arg', 'arg_idx', 'verb_idx', 'structure',
       'decomp_version', 'Pred.Lemma', 'modified_sentence', 'Arg.Tokens.Begin',
       'combined_labels', 'VN_mappings', 'labels', 'awareness',
       'change_of_location', 'change_of_state', 'change_of_possession',
       'existed_after', 'existed_before', 'existed_during', 'instigation',
       'sentient', 'volition'],
      dtype='object')

In [7]:
roles_wide["gram_position"] = roles_wide["Gram.Func"].replace("iobj", "dobj")

pd.DataFrame(roles_wide[
    ["decomp_version", "structure", "Split", "gram_position"]].value_counts()
             ).reset_index().pivot_table(
                 index="Split", columns=["decomp_version", "structure", "gram_position"], values="count"
             )


decomp_version          V1                                       V2          \
structure      active_full         passive_full         active_full           
gram_position         dobj   nsubj         dobj   nsubj        dobj   nsubj   
Split                                                                         
dev                  477.0   490.0        182.0   183.0       231.0   507.0   
test                 488.0   477.0        177.0   200.0       249.0   517.0   
train               3662.0  4097.0       1544.0  1520.0      1627.0  3898.0   

decomp_version                       
structure      passive_full          
gram_position          dobj   nsubj  
Split                                
dev                   221.0   277.0  
test                  241.0   263.0  
train                1514.0  1864.0

In [6]:
pd.DataFrame(roles_wide[
    ["decomp_version", "structure"]].value_counts()
             )


count
decomp_version structure          
V1             active_full    9691
V2             active_full    7029
               passive_full   4380
V1             passive_full   3806